In [8]:
import sys
from pathlib import Path

# Project root is current directory
project_root = Path.cwd()
sys.path.append(str(project_root))

# Add metric_pipeline for internal imports
metric_pipeline_path = project_root / "rag_modules_src" / "metric_pipeline"
sys.path.insert(0, str(metric_pipeline_path))

print(f"✓ Project root: {project_root}")

✓ Project root: /Users/vishaknair/Documents/FinSights/ModelPipeline/finrag_ml_tg1


In [9]:
from loaders.ml_config_loader import MLConfig

# Import MetricPipeline (with metric_pipeline in path from Cell 1)
from rag_modules_src.metric_pipeline.src.pipeline import MetricPipeline

# Import BedrockClient directly (skip __init__.py)
import sys
sys.path.insert(0, str(project_root / "rag_modules_src" / "synthesis_pipeline"))
from bedrock_client import BedrockClient

print("✓ All imports successful")

✓ All imports successful


In [10]:
# Load configuration + AWS credentials
config = MLConfig()
bedrock_client = config.get_bedrock_client()

print("✓ Configuration loaded")
print("✓ Bedrock client initialized")

[DEBUG] ✓ AWS credentials loaded from aws_credentials.env
✓ Configuration loaded
✓ Bedrock client initialized


In [11]:
# Initialize metric pipeline (KPI fact table connector)
base_path = project_root / "rag_modules_src"

metric_pipeline = MetricPipeline(
    data_path=str(base_path / "metric_pipeline" / "data" / "downloaded_data.json"),
    company_dim_path=str(project_root / "data_cache" / "dimensions" / "finrag_dim_companies_21.parquet")
)

print("✓ Metric pipeline initialized")

✓ FilterExtractor initialized with 21 companies
✓ Loaded 527 metric records
✓ Unique tickers: 2
✓ Year range: 2010-2025
✓ Metric pipeline initialized


In [12]:
# Initialize LLM client (Bedrock Claude)
llm_config = config.cfg.get('rag_orchestrator', {}).get('llm', {})

llm_client = BedrockClient(
    region=llm_config.get('region', 'us-east-1'),
    model_id=llm_config.get('model_id', 'anthropic.claude-sonnet-4-20250514'),
    max_tokens=llm_config.get('max_tokens', 4096),
    temperature=llm_config.get('temperature', 0.7)
)

print("✓ LLM client initialized")

✓ LLM client initialized


In [13]:
# Define user query
user_query = "What were Microsoft's and NVIDIA's total assets and revenue from 2021 to 2023?"

print(f"Query: {user_query}")

Query: What were Microsoft's and NVIDIA's total assets and revenue from 2021 to 2023?


In [14]:
# Extract from KPI fact tables
result = metric_pipeline.process(user_query)

# Check if successful
if not result.get('success'):
    print(f"❌ No analytical data: {result.get('reason')}")
    print("\nFilters extracted:")
    print(f"  Tickers: {result.get('extracted_filters', {}).get('tickers', [])}")
    print(f"  Years: {result.get('extracted_filters', {}).get('years', [])}")
    print(f"  Metrics: {result.get('extracted_filters', {}).get('metrics', [])}")
else:
    print("✓ Data extracted successfully")
    print(f"  Data points: {result.get('count', 0)}")
    print(f"  Coverage: {result.get('stats', {}).get('coverage_pct', 0):.1f}%")

✓ Data extracted successfully
  Data points: 8
  Coverage: 66.7%


In [15]:
def format_value_compact(value: float) -> str:
    """Format financial values compactly."""
    if abs(value) >= 1_000_000_000:
        return f"${value/1_000_000_000:.1f}B"
    elif abs(value) >= 1_000_000:
        return f"${value/1_000_000:.1f}M"
    elif abs(value) >= 1_000:
        return f"${value/1_000:.0f}K"
    else:
        return f"${value:.0f}"

def format_analytical_compact(raw_result: dict) -> str:
    """Format fact table results as ultra-compact string."""
    data = raw_result.get('data', [])
    if not data:
        return ""
    
    from collections import defaultdict
    grouped = defaultdict(lambda: defaultdict(dict))
    
    for item in data:
        if item.get('found'):
            ticker = item['ticker']
            year = item['year']
            metric = item['metric']
            value = item['value']
            
            # Shorten metric names
            metric_short = (metric.replace('income_stmt_', '')
                                 .replace('balance_sheet_', '')
                                 .replace('cash_flow_', '')
                                 .replace('_', ''))
            grouped[ticker][year][metric_short] = value
    
    lines = []
    for ticker, years_data in sorted(grouped.items()):
        for year, metrics in sorted(years_data.items()):
            metrics_str = ', '.join([f"{k}={format_value_compact(v)}" for k, v in metrics.items()])
            lines.append(f"{ticker} {year}: {metrics_str}")
    
    return '\n'.join(lines)

print("✓ Helper functions defined")

✓ Helper functions defined


In [16]:
# Format as compact string for LLM
compact_data = format_analytical_compact(result)

print("Compact KPI data (→ LLM):")
print("-" * 70)
print(compact_data)
print("-" * 70)

token_estimate = len(compact_data) // 4
print(f"\nToken estimate: ~{token_estimate} tokens")

Compact KPI data (→ LLM):
----------------------------------------------------------------------
MSFT 2021: Total Assets=$333.8B
MSFT 2022: Total Assets=$364.8B
MSFT 2023: Total Assets=$412.0B
NVDA 2021: Total Assets=$28.8B
NVDA 2022: Total Assets=$44.2B, Revenue=$26.9B
NVDA 2023: Total Assets=$41.2B, Revenue=$27.0B
----------------------------------------------------------------------

Token estimate: ~55 tokens


In [17]:
# Generate final answer using Claude
final_answer = llm_client.generate_response(
    user_query=user_query,
    analytical_results=compact_data,
    rag_context=[]
)

print("="*70)
print("FINAL ANSWER")
print("="*70)
print(final_answer)
print("="*70)

FINAL ANSWER
Based on the provided financial data, here's a summary of Microsoft's and NVIDIA's total assets and revenue from 2021 to 2023:

Microsoft (MSFT):
2021: Total Assets = $333.8 billion
2022: Total Assets = $364.8 billion
2023: Total Assets = $412.0 billion

Revenue figures for Microsoft are not provided in the given data.

NVIDIA (NVDA):
2021: Total Assets = $28.8 billion (Revenue not provided)
2022: Total Assets = $44.2 billion, Revenue = $26.9 billion
2023: Total Assets = $41.2 billion, Revenue = $27.0 billion

It's important to note that while we have total assets data for both companies across all three years, revenue figures are only provided for NVIDIA for 2022 and 2023. Microsoft's revenue data and NVIDIA's 2021 revenue are not included in the given information.


In [18]:
# Display extracted filters (for S3 metadata filtering)
filters = result.get('filters', {})

print("📌 Extracted Filters (for S3 metadata):")
print(f"  Companies: {filters.get('tickers', [])}")
print(f"  Years: {filters.get('years', [])}")

metrics_short = [
    m.replace('income_stmt_', '').replace('balance_sheet_', '').replace('cash_flow_', '') 
    for m in filters.get('metrics', [])
]
print(f"  Metrics: {metrics_short}")

📌 Extracted Filters (for S3 metadata):
  Companies: ['MSFT', 'NVDA']
  Years: [2021, 2022, 2023]
  Metrics: ['Total Assets', 'Revenue']
